RFB，FSSD，RetinaNet，FCN
# 4.Focal Loss与RetinateNet
YOLO的精度不够高，这是因为它做的是稠密分类，核心问题是稠密proposal中前景和背景的极度不平衡。比如PASCAL VOC数据集中，每张图片上标注的目标可能也就几个，但是YOLO V2最后一层的输出是13*13*5,也就是845个候选目标，大量的负样本在loss中占据了很大比重，使得有用的loss不能回传回来。基于此，作者将经典的交叉熵损失做了变形，给那些易于被分类的简单例子小的权重，给不易区分的难例更大的权重。同时，作者提出了一个新的one-stage的检测器RetinaNet，达到了速度和精度很好地trade-off。

## 4.1.交叉熵损失
Focal Loss从交叉熵损失而来，二分类的交叉熵损失如下:
$$CE(p,y)=\begin{cases}
-log(p) & y=1\\
-log(1-p) & otherwise
\end{cases}$$
对应的，多分类的交叉熵损失是这样的：
$$CE(p,y)=-log(p_y)$$
![images](../Results/01/02_02_01_004.png)<br/>
如上图所示，蓝色线为交叉熵损失函数随着pt变化的曲线(pt意为ground truth，是标注类别所对应的概率)。可以看到，当概率大于.5，即认为是易分类的简单样本时，值仍然较大。这样，很多简单样本累加起来，就很可能盖住那些稀少的不易正确分类的类别

## 4.2.Focal Loss损失
为了改善类别样本分布不均衡的问题，已经有人提出了使用加上权重的交叉熵损失：
$$CE(p)=-\alpha_tlog(p_t)$$
即用参数$\alpha_t$来平衡，这组参数可以是超参数，也可以由类别的比例倒数决定。作者将其作为比较的baseline，提出了一个自适应调节的权重，即Focal Loss，定义如下：
$$FL(p_t)=-(1-p_t)^{\gamma}log(p_t)$$
在实际实验中，作者使用的是加权之后的Focal Loss，作者发现这样能够带来些微的性能提升。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)
    mask = torch.Tensor(*size).fill_(0)
    index = index.view(*view)
    ones = 1.
    if isinstance(index, Variable):
        ones = Variable(torch.Tensor(index.size()).fill_(1))
        mask = Variable(mask, volatile=index.volatile)
    return mask.scatter_(1, index, ones)
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
    def forward(self, input, target):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input)
        logit = logit.clamp(self.eps, 1. - self.eps)
        loss = -1 * y * torch.log(logit) # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss
        return loss.sum()

## 4.3.RetinaNet
利用Focal Loss，基于ResNet和Feature Pyramid Net(FPN)设计了一种新的one-stage检测框架，命名为RetinaNet

